<a href="https://colab.research.google.com/github/NadaSOsman/MobilityPred-COVID-Restrictions/blob/master/Model_extra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import zipfile
from os import listdir
from os.path import isfile, join
import gzip
import shutil

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

local_download_path = os.path.expanduser('/root/data')
try:
	os.makedirs(local_download_path)
except: pass

file_list = drive.ListFile(
	{'q': "'1aLmDCh-VHHvZExBxHyesB4W2j9KAnRlt' in parents"}).GetList()

for f in file_list:
	# 3. Create & download by id.
	print('title: %s, id: %s' % (f['title'], f['id']))
	fname = os.path.join(local_download_path, f['title'])
	print('downloading to {}'.format(fname))
	f_ = drive.CreateFile({'id': f['id']})
	f_.GetContentFile(fname)
 

import numpy as np
import tensorflow as tf
import random as rn
import sys

import os
os.environ['PYTHONHASHSEED'] = '0'

np.random.seed(1442)
rn.seed(12435)

from tensorflow.compat.v1.keras import backend as K

tf.random.set_seed(1234)

session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
K.set_session(sess)

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, LSTM, Convolution2D, Flatten, Concatenate, Reshape
from keras.callbacks import LearningRateScheduler
import csv
import random
import gc
from keras import optimizers

INPUT_WIN_SIZE = 10
OUTPUT_WIN_SIZE = 1
MOBILITY_CATEGORIES = 6
TRAINIG_DATA_SAMPLES = 51
TESTING_DATA_SAMPLES = 9
SAMPLE_SIZE = 164
BATCH_SIZE = 44
CONTROL_MEASURES = 8
INFECTION_FEATURES = 4


with open("/root/data/training_extra.csv", 'r', encoding="utf8") as f:
  reader = csv.reader(f, delimiter=',')
  headers = next(reader)
  training_data = np.array(list(reader)).astype(float)
f.close()

with open("/root/data/testing_extra.csv", 'r', encoding="utf8") as f:
  reader = csv.reader(f, delimiter=',')
  headers = next(reader)
  testing_data = np.array(list(reader)).astype(float)
f.close()

def build_model():
  inputs = []
  outputs = []
  autoregressives = []
  for i in range(0, MOBILITY_CATEGORIES):
    input = Input(shape=(INPUT_WIN_SIZE,), dtype='int32')
    """fc1 = Dense(32, activation='linear')(input)
    d1 = Dropout(0.2)(fc1)
    fc2 = Dense(16, activation='linear')(d1)
    d2 = Dropout(0.2)(fc2)
    fc3 = Dense(8, activation='linear')(d2)
    d3 = Dropout(0.2)(fc3)
    output = Dense(1, activation='linear')(d3)"""
    output = Dense(1, activation='linear')(input)
    
    inputs.append(input)
    autoregressives.append(output)

  mobility = Input(shape=(INPUT_WIN_SIZE,MOBILITY_CATEGORIES,3), dtype='float32')
  control = Input(shape=(INPUT_WIN_SIZE,CONTROL_MEASURES,2), dtype='float32')
  infections = Input(shape=(INPUT_WIN_SIZE,INFECTION_FEATURES,1), dtype='float32')
  holidays = Input(shape=(INPUT_WIN_SIZE,), dtype='float32')

  inputs.append(mobility)
  inputs.append(control)
  inputs.append(infections)
  inputs.append(holidays)

  c1 = Convolution2D(8, (3, MOBILITY_CATEGORIES), activation='linear', data_format='channels_last')(mobility)
  c2 = Convolution2D(8, (5, MOBILITY_CATEGORIES), activation='linear', data_format='channels_last')(mobility)
  c3 = Convolution2D(8, (7, MOBILITY_CATEGORIES), activation='linear', data_format='channels_last')(mobility)

  c4 = Convolution2D(8, (3, CONTROL_MEASURES), activation='linear', data_format='channels_last')(control)
  c5 = Convolution2D(8, (5, CONTROL_MEASURES), activation='linear', data_format='channels_last')(control)
  c6 = Convolution2D(8, (7, CONTROL_MEASURES), activation='linear', data_format='channels_last')(control)

  c7 = Convolution2D(8, (3, INFECTION_FEATURES), activation='linear', data_format='channels_last')(infections)
  c8 = Convolution2D(8, (5, INFECTION_FEATURES), activation='linear', data_format='channels_last')(infections)
  c9 = Convolution2D(8, (7, INFECTION_FEATURES), activation='linear', data_format='channels_last')(infections)
  
  r1 = Reshape((8,INPUT_WIN_SIZE-2))(c1)
  r2 = Reshape((8,INPUT_WIN_SIZE-4))(c2)
  r3 = Reshape((8,INPUT_WIN_SIZE-6))(c3)
  r4 = Reshape((8,INPUT_WIN_SIZE-2))(c4)
  r5 = Reshape((8,INPUT_WIN_SIZE-4))(c5)
  r6 = Reshape((8,INPUT_WIN_SIZE-6))(c6)
  r7 = Reshape((8,INPUT_WIN_SIZE-2))(c7)
  r8 = Reshape((8,INPUT_WIN_SIZE-4))(c8)
  r9 = Reshape((8,INPUT_WIN_SIZE-6))(c9)
  
  l1 = LSTM(32, return_sequences=True, dropout=0.2)(r1)
  l2 = LSTM(32, return_sequences=True, dropout=0.2)(r2)
  l3 = LSTM(32, return_sequences=True, dropout=0.2)(r3)
  l4 = LSTM(32, return_sequences=True, dropout=0.2)(r4)
  l5 = LSTM(32, return_sequences=True, dropout=0.2)(r5)
  l6 = LSTM(32, return_sequences=True, dropout=0.2)(r6)
  l7 = LSTM(32, return_sequences=True, dropout=0.2)(r7)
  l8 = LSTM(32, return_sequences=True, dropout=0.2)(r8)
  l9 = LSTM(32, return_sequences=True, dropout=0.2)(r9)

  f1 = Flatten()(l1)
  f2 = Flatten()(l2)
  f3 = Flatten()(l3)
  f4 = Flatten()(l4)
  f5 = Flatten()(l5)
  f6 = Flatten()(l6)
  f7 = Flatten()(l7)
  f8 = Flatten()(l8)
  f9 = Flatten()(l9)

  """fc1 = Dense(16, activation='linear')(f1)
  fc2 = Dense(16, activation='linear')(f2)
  fc3 = Dense(16, activation='linear')(f3)
  fc4 = Dense(16, activation='linear')(f4)
  fc5 = Dense(16, activation='linear')(f5)
  fc6 = Dense(16, activation='linear')(f6)

  d1 = Dropout(0.2)(fc1)
  d2 = Dropout(0.2)(fc2)
  d3 = Dropout(0.2)(fc3)
  d4 = Dropout(0.2)(fc4)
  d5 = Dropout(0.2)(fc5)
  d6 = Dropout(0.2)(fc6)

  fc7 = Dense(8, activation='linear')(d1)
  fc8 = Dense(8, activation='linear')(d2)
  fc9 = Dense(8, activation='linear')(d3)
  fc10 = Dense(8, activation='linear')(d4)
  fc11 = Dense(8, activation='linear')(d5)
  fc12 = Dense(8, activation='linear')(d6)

  d7 = Dropout(0.2)(fc7)
  d8 = Dropout(0.2)(fc8)
  d9 = Dropout(0.2)(fc9)
  d10 = Dropout(0.2)(fc10)
  d11 = Dropout(0.2)(fc11)
  d12 = Dropout(0.2)(fc12)"""

  for autoregressive in autoregressives:
    #fc = Concatenate()([d7, d8, d9, d10, d11, d12, autoregressive])
    fc = Concatenate()([f1, f2, f3, f4, f5, f6, f7, f8, f9, holidays, autoregressive])
    output = output = Dense(1, activation='linear')(fc)
    outputs.append(output)

  model = Model(inputs=inputs, outputs=outputs)
  model.compile(loss='mean_squared_error', optimizer=optimizers.Adam(learning_rate=0.01))

  print(model.summary())
  return model

def step_decay(epoch, lr):
	lrate = lr
	if(epoch%1000 == 0 and epoch != 0):
		#lrate = lr/(2.0-min((epoch/100.0 - 1)*0.1, 1.0))
		lrate = lr*0.7
	print(lrate)
	return lrate

global current_training_indeces
def generator(training):
  global current_training_indeces
  if (training == 1):
    start_batch = random.randint(0,TRAINIG_DATA_SAMPLES)
    indeces = [i*SAMPLE_SIZE for i in range(start_batch, min(start_batch+BATCH_SIZE, TRAINIG_DATA_SAMPLES))]
    diff = BATCH_SIZE-len(indeces)
    for i in range (0, diff):
      indeces.append(i*SAMPLE_SIZE)
    current_training_indeces = indeces[:]
    data = np.transpose(training_data)
  elif (training == 2):
    indeces = []
    for i in range(0, TRAINIG_DATA_SAMPLES):
      if (i*SAMPLE_SIZE not in current_training_indeces):
        indeces.append(i*SAMPLE_SIZE)
    data = np.transpose(training_data)
  else:
    indeces = [i*SAMPLE_SIZE for i in range(0,TESTING_DATA_SAMPLES)]
    data = np.transpose(testing_data)

  time_series_input = [[] for i in range(0, MOBILITY_CATEGORIES+4)]
  time_series_output = [[] for i in range(0, MOBILITY_CATEGORIES)]
  for index in indeces:
    i = index
    while i < index+SAMPLE_SIZE-(INPUT_WIN_SIZE+OUTPUT_WIN_SIZE):
      for c in range(0, MOBILITY_CATEGORIES):
        time_series_input[c].append(np.reshape(data[c*3+CONTROL_MEASURES*2][i:i+INPUT_WIN_SIZE],(INPUT_WIN_SIZE,)))
        time_series_output[c].append(np.reshape(data[c*3+CONTROL_MEASURES*2][i+INPUT_WIN_SIZE:i+INPUT_WIN_SIZE+OUTPUT_WIN_SIZE],(OUTPUT_WIN_SIZE,)))
      time_series_input[MOBILITY_CATEGORIES].append(np.reshape(np.transpose(data[CONTROL_MEASURES*2:-INFECTION_FEATURES-1][:,i:i+INPUT_WIN_SIZE]),(INPUT_WIN_SIZE,MOBILITY_CATEGORIES,3)))
      time_series_input[MOBILITY_CATEGORIES+1].append(np.reshape(np.transpose(data[:CONTROL_MEASURES*2][:,i:i+INPUT_WIN_SIZE]),(INPUT_WIN_SIZE,CONTROL_MEASURES,2)))
      time_series_input[MOBILITY_CATEGORIES+2].append(np.reshape(np.transpose(data[CONTROL_MEASURES*2+MOBILITY_CATEGORIES*3:-1][:,i:i+INPUT_WIN_SIZE]),(INPUT_WIN_SIZE,INFECTION_FEATURES,1)))
      time_series_input[MOBILITY_CATEGORIES+3].append(np.reshape(np.transpose(data[-1][i:i+INPUT_WIN_SIZE]),(INPUT_WIN_SIZE,)))
      i += 1

  for c in range(0, MOBILITY_CATEGORIES):
     time_series_input[c] = np.asarray(time_series_input[c])
     time_series_output[c] = np.asarray(time_series_output[c])
  #print(time_series_input[MOBILITY_CATEGORIES+1][0])
  time_series_input[MOBILITY_CATEGORIES] = np.asarray(time_series_input[MOBILITY_CATEGORIES])
  time_series_input[MOBILITY_CATEGORIES+1] = np.asarray(time_series_input[MOBILITY_CATEGORIES+1])
  time_series_input[MOBILITY_CATEGORIES+2] = np.asarray(time_series_input[MOBILITY_CATEGORIES+2])
  time_series_input[MOBILITY_CATEGORIES+3] = np.asarray(time_series_input[MOBILITY_CATEGORIES+3])

  #print(time_series_input[0].shape, time_series_input[-1].shape)
  return time_series_input, time_series_output

def validation_batch_generator():
	while True:
		gc.collect()
		time_series_input, time_series_output = generator(2)
		yield time_series_input, time_series_output

def training_batch_generator():
	while True:
		gc.collect()
		time_series_input, time_series_output = generator(1)
		yield time_series_input, time_series_output

def train_model(model):
  lrate = LearningRateScheduler(step_decay)
  model.fit_generator(generator = training_batch_generator(), steps_per_epoch=1, epochs=2000, verbose=1, validation_data=validation_batch_generator(), validation_steps=1, callbacks=[lrate])
  return model


model = build_model()
model = train_model(model)

time_series_input, time_series_output = generator(3)
pred = (np.asarray(model.predict(time_series_input)))
true = (np.asarray(time_series_output))

pred_file = open("/root/data/pred.csv", 'w')
true_file = open("/root/data/true.csv", 'w')

for i in range(pred.shape[1]):
  p = ""
  t = ""
  for j in range(pred.shape[0]):
    p += str(pred[j][i][0])+","
    t += str(true[j][i][0])+","
  pred_file.write(p[:-1]+"\n")
  true_file.write(t[:-1]+"\n")
pred_file.close()
true_file.close()

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

local_download_path = os.path.expanduser('~/data')
try:
	os.makedirs(local_download_path)
except: pass

uploaded = drive.CreateFile({'title': 'pred.csv'})
uploaded.SetContentFile('/root/data/pred.csv')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

uploaded = drive.CreateFile({'title': 'true.csv'})
uploaded.SetContentFile('/root/data/true.csv')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Streaming output truncated to the last 5000 lines.
Epoch 335/2000
1/1 [==============================] - 1s 743ms/step - loss: 675.8292 - dense_6_loss: 45.5183 - dense_7_loss: 107.9766 - dense_8_loss: 409.7816 - dense_9_loss: 33.1030 - dense_10_loss: 71.4085 - dense_11_loss: 8.0411 - val_loss: 685.4303 - val_dense_6_loss: 50.2129 - val_dense_7_loss: 127.4598 - val_dense_8_loss: 393.1026 - val_dense_9_loss: 36.7967 - val_dense_10_loss: 69.7134 - val_dense_11_loss: 8.1447
0.009999999776482582
Epoch 336/2000
1/1 [==============================] - 1s 788ms/step - loss: 638.3649 - dense_6_loss: 44.4377 - dense_7_loss: 105.7447 - dense_8_loss: 376.6501 - dense_9_loss: 32.6286 - dense_10_loss: 70.6704 - dense_11_loss: 8.2333 - val_loss: 468.8441 - val_dense_6_loss: 33.0791 - val_dense_7_loss: 74.1394 - val_dense_8_loss: 253.2123 - val_dense_9_loss: 27.2092 - val_dense_10_loss: 75.1287 - val_dense_11_loss: 6.0754
0.009999999776482582
Epoch 337/2000
1/1 [==============================] - 1s 796